In [1]:
# ループ学習した各iterationでのレーティングバトルを実行

In [2]:
from bson import ObjectId
import numpy as np
from pokeai.util import json_load, pickle_load, ROOT_DIR, DATASET_DIR, yaml_dump
from collections import Counter
import json
import yaml
import os
import copy

In [3]:
file_base_dir = r"D:\dev\pokeai\pokeai\experiment\team\rl\rl_loop_210227_1"
tag_prefix = "rl_loop_210227_1_"

In [4]:
with open(os.path.join(file_base_dir, f"{tag_prefix}run.bat"), "r") as f:
    bat_lines = f.readlines()

In [5]:
bat_lines

['\n',
 'python -m pokeai.ai.generate_party rl_loop_210227_1_hc_0 -n 871 -r finalgoodmove3vs3\n',
 '\n',
 'python -m pokeai.ai.generate_party rl_loop_210227_1_allonce_0 --all_pokemon_once -r finalgoodmove3vs3\n',
 'python -m pokeai.ai.generic_move_model.rl_train D:\\dev\\pokeai\\pokeai\\experiment\\team\\rl\\rl_loop_210227_1\\rl_loop_210227_1_0_train.yaml \n',
 'python -m pokeai.ai.party_feature.hillclimb_party D:\\dev\\pokeai\\pokeai\\experiment\\team\\rl\\rl_loop_210227_1\\rl_loop_210227_1_0_hc.yaml\n',
 'rem {"iternum": 0, "trainer_id": "6039caccaf28292a4eae380a", "train_party_tags": ["rl_loop_210227_1_allonce_0", "rl_loop_210227_1_hc_0"], "gen_party_tag": "rl_loop_210227_1_hc_1"}\n',
 '\n',
 'python -m pokeai.ai.generate_party rl_loop_210227_1_allonce_1 --all_pokemon_once -r finalgoodmove3vs3\n',
 'python -m pokeai.ai.generic_move_model.rl_train D:\\dev\\pokeai\\pokeai\\experiment\\team\\rl\\rl_loop_210227_1\\rl_loop_210227_1_1_train.yaml --initialize_by_trainer 6039caccaf28292a4ea

In [6]:
configs = []
for line in bat_lines:
    if line.startswith("rem "):
        configs.append(json.loads(line[4:]))

In [7]:
configs

[{'iternum': 0,
  'trainer_id': '6039caccaf28292a4eae380a',
  'train_party_tags': ['rl_loop_210227_1_allonce_0', 'rl_loop_210227_1_hc_0'],
  'gen_party_tag': 'rl_loop_210227_1_hc_1'},
 {'iternum': 1,
  'trainer_id': '6039caccaf28292a4eae380b',
  'train_party_tags': ['rl_loop_210227_1_allonce_1', 'rl_loop_210227_1_hc_1'],
  'gen_party_tag': 'rl_loop_210227_1_hc_2'},
 {'iternum': 2,
  'trainer_id': '6039caccaf28292a4eae380c',
  'train_party_tags': ['rl_loop_210227_1_allonce_2', 'rl_loop_210227_1_hc_2'],
  'gen_party_tag': 'rl_loop_210227_1_hc_3'},
 {'iternum': 3,
  'trainer_id': '6039caccaf28292a4eae380d',
  'train_party_tags': ['rl_loop_210227_1_allonce_3', 'rl_loop_210227_1_hc_3'],
  'gen_party_tag': 'rl_loop_210227_1_hc_4'},
 {'iternum': 4,
  'trainer_id': '6039caccaf28292a4eae380e',
  'train_party_tags': ['rl_loop_210227_1_allonce_4', 'rl_loop_210227_1_hc_4'],
  'gen_party_tag': 'rl_loop_210227_1_hc_5'},
 {'iternum': 5,
  'trainer_id': '6039caccaf28292a4eae380f',
  'train_party_tags'

In [8]:
rate_ids = [ObjectId() for _ in range(len(configs))]

In [9]:
import subprocess

In [10]:
for config, rate_id in zip(configs, rate_ids):
    cmd = f"python -m pokeai.ai.generic_move_model.rl_rating_battle {config['trainer_id']} {','.join(config['train_party_tags'])} --rate_id {rate_id}"
    print(cmd)
    subprocess.check_call(cmd, shell=True)

python -m pokeai.ai.generic_move_model.rl_rating_battle 6039caccaf28292a4eae380a rl_loop_210227_1_allonce_0,rl_loop_210227_1_hc_0 --rate_id 6049e2cd3410a8cbf644a642
python -m pokeai.ai.generic_move_model.rl_rating_battle 6039caccaf28292a4eae380b rl_loop_210227_1_allonce_1,rl_loop_210227_1_hc_1 --rate_id 6049e2cd3410a8cbf644a643
python -m pokeai.ai.generic_move_model.rl_rating_battle 6039caccaf28292a4eae380c rl_loop_210227_1_allonce_2,rl_loop_210227_1_hc_2 --rate_id 6049e2cd3410a8cbf644a644
python -m pokeai.ai.generic_move_model.rl_rating_battle 6039caccaf28292a4eae380d rl_loop_210227_1_allonce_3,rl_loop_210227_1_hc_3 --rate_id 6049e2cd3410a8cbf644a645
python -m pokeai.ai.generic_move_model.rl_rating_battle 6039caccaf28292a4eae380e rl_loop_210227_1_allonce_4,rl_loop_210227_1_hc_4 --rate_id 6049e2cd3410a8cbf644a646
python -m pokeai.ai.generic_move_model.rl_rating_battle 6039caccaf28292a4eae380f rl_loop_210227_1_allonce_5,rl_loop_210227_1_hc_5 --rate_id 6049e2cd3410a8cbf644a647
python -m 

In [11]:
iter_ids = []
for config, rate_id in zip(configs, rate_ids):
    c = copy.deepcopy(config)
    c["rate_id"] = str(rate_id)
    iter_ids.append(c)
with open(os.path.join(file_base_dir, f"iter_ids.json"), "w") as f:
    json.dump(iter_ids, f)